In [1]:
%matplotlib inline

import sys
import os
import numpy as np
import json
import pandas as pd
import traceback

sys.path.append("../../../") # add results folder as a python path 
from viz import *
from analyze import *

In [2]:
path = os.path.dirname("./data2/BOHB/")
target_dirs = os.listdir(path)

In [3]:
def getTrialResult(result):
    cum_exec_time = []
    cum_opt_time = []
    cum_first = True

    exec_times = []
    opt_times = []
    epochs = []
    model_idx = []
    errors = []
    accuracies = []

    for r in result:
        dict_r = json.loads(r)
        exec_time = dict_r[3]['info']['elapsed time']
        opt_time = dict_r[2]['finished'] - dict_r[2]['submitted']

        if cum_first is True:
            cum_exec_time_last_value = 0.0
            cum_opt_time_last_value = 0.0
            cum_first = False

        if cum_exec_time_last_value <= 86400:
            exec_times.append(exec_time)
            opt_times.append(opt_time)
            cum_exec_time.append((cum_exec_time_last_value + exec_time))
            cum_opt_time.append((cum_opt_time_last_value + opt_time))
            epochs.append(dict_r[1])
            model_idx.append(dict_r[0])
            errors.append(dict_r[3]['loss'])
            accuracies.append(dict_r[3]['info']['test accuracy'])

            cum_exec_time_last_value = cum_exec_time[len(cum_exec_time)-1]
            cum_opt_time_last_value = cum_opt_time[len(cum_opt_time)-1]
            
            dict_result = dict([("cum_exec_time", cum_exec_time),
                        ("exec_time", exec_times),
                        ("opt_time", opt_times),
                        ("model_index", model_idx),
                        ("cum_opt_time", cum_opt_time),
                        ("error", errors),
                        ("accuracy", accuracies),
                        ("epoch", epochs)])
        else:
            continue
        
    return dict_result

In [6]:
def getTrialConfig(config):
    config_dict = dict()
    
    for c in config:
        dict_c = json.loads(c)
        target = dict_c[2]
        config_dict[str(dict_c[0])] = target
        
    return config_dict

In [20]:
def addModelInfo(trial_result, trial_config):
    model_info_list = []
    for i in range(len(trial_result['model_index'])):
        model_idx = str(trial_result['model_index'][i])
        model_info = trial_config[model_idx]
        model_info_list.append(model_info)
    trial_result['model_info'] = model_info_list

In [21]:
bohb_results = dict()
bohb_configs = dict()

for d in target_dirs:
    trial_num = d[5:]
    
    with open(path+'/'+d+'/'+'results.json') as rf:
        result = rf.readlines()

    result = [x.strip() for x in result]
    result_dict=getTrialResult(result)
    
    with open(path+'/'+d+'/'+'configs.json') as rf:
        config = rf.readlines()
        
    config = [x.strip() for x in config]
#     bohb_configs[trial_num]=getTrialConfig(config)
    config_dict=getTrialConfig(config)
    
    addModelInfo(result_dict, config_dict)
    
    bohb_results[trial_num]=result_dict

In [22]:
bohb_results["0"]

{'cum_exec_time': [14.483183677777777,
  27.0170459,
  45.73430321111111,
  73.81091332222222,
  111.33878845555554,
  131.13278465555555,
  147.60565076666666,
  184.0083484333333,
  269.5342156555555,
  382.1178410555555,
  491.32593405555554,
  747.9035357222222,
  1085.6544119222222,
  1196.0382686222222,
  1249.0215872555555,
  1331.5920878555555,
  1662.7436579555556,
  1997.5829550555557,
  2556.517204255556,
  2736.502433055556,
  2771.543804722223,
  2818.7078345111117,
  2862.9048094000004,
  2888.0370823111116,
  2916.6077106333337,
  2945.218350766667,
  2979.0234022222226,
  3010.9024508888892,
  3059.034247988889,
  3164.158362988889,
  3249.870247955556,
  3394.265639255556,
  3709.637984255556,
  3808.9764355555562,
  3933.454141822223,
  4011.2317581555562,
  4309.2471120555565,
  4571.180362955556,
  4891.243609955556,
  5237.211695955556,
  5260.89265228889,
  5381.761145177778,
  5430.818572188889,
  5507.311351422223,
  5572.212704644446,
  5672.83847218889,
  5721

In [24]:
with open('data2_result.json', 'w') as outfile:
    json.dump(bohb_results, outfile)